In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import yfinance as yf
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
# top 40 dax companies 
dax_assets = {
               'DAX': '^GDAXI',
               'Linde': 'LIN',
               'SAP': 'SAP',
               'Deutsche Telekom': 'DTE.DE',
               'Volkswagen': 'VOW3.DE',
               'Siemens': 'SIE.DE',
               'Merck': 'MRK.DE',
               'Airbus': 'AIR.PA',
               'Mercedes Benz': 'MBG.DE', 
               'Bayer': 'BAYZF',
               'BMW': 'BMW.DE',
               'Siemens Healthineers': 'SHL.DE',
               'Deutsche Post': 'DPW.DE',
               'BASF': 'BAS.DE',
               'Münchner Rück': 'MUV2.DE',
               'Infineon': 'IFX.DE',
               'Deutsche Börse': 'DB1:DE',
               'RWE': 'RWE.DE',
               'Henkel': 'HEN3.DE',
               'Adidas': 'ADS.DE',
               'Sartorius': 'SRT.DE',
               'Beiersdorf': 'BEI.DE',
               'Porsche': 'PAH3.DE',
               'E.ON': 'EOAN.DE',
               'Deutsche Bank': 'DB',
               'Vonovia': 'VNA.DE',
               'Fresenius': 'FRE.DE',
               'Symrise': 'SY1.DE',
               'Continental': 'CON.DE',
               'Delivery Hero': 'DHER.F',
               'Brenntag': 'BNR.DE',
               'Qiagen': 'QGEN',
               'Fresenius Medical Care': 'FMS',
               'Siemens Energy': 'ENR.F',
               'HeidelbergCement': 'HEI.DE',
               'Puma': 'PUM.DE',
               'MTU Aero Engines': 'MTX.DE',
               'Covestro': '1COV.F',
               'Zalando': 'ZAL.DE',
               'HelloFresh': 'HFG.DE'
            }

In [3]:
dax_top_40_companies = list(dax_assets.keys())#[1:]
dax_top_40_tickername = list(dax_assets.values())#[1:]

In [4]:
start_date = '2019-01-01'
end_date = '2022-09-15'
na_percentage = 0.6 # at least x percent rows must be none-nas

# download DAX top40
df = yf.download(dax_top_40_tickername, 
                    start=start_date, 
                    end=end_date,
                    progress=True)
# drop columns
df = df['Close']
# rename columns
df.columns = dax_top_40_companies
# make sure the index is datetime format
df.index = pd.to_datetime(df.index)
# drop nas
datetimeFormat = '%Y-%m-%d'
time_delta = datetime.strptime(end_date, datetimeFormat) - datetime.strptime(start_date,datetimeFormat)
df.dropna(axis=1, thresh=int(time_delta.days * na_percentage), inplace=True)
# show data
df.head()

[*********************100%***********************]  40 of 40 completed

1 Failed download:
- DB1:DE: No data found, symbol may be delisted


,DAX,Linde,SAP,Deutsche Telekom,Volkswagen,Siemens,Merck,Airbus,Mercedes Benz,Bayer,...,Brenntag,Qiagen,Fresenius Medical Care,Siemens Energy,HeidelbergCement,Puma,MTU Aero Engines,Covestro,Zalando,HelloFresh
Date,,,,,,,,,,,,,,,,,,,,,
2018-12-31,NaN,NaN,NaN,NaN,69.500000,NaN,NaN,NaN,NaN,8.15,...,NaN,99.550003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-02,42.900002,184.399994,84.169998,60.720001,69.599998,91.860001,69.739998,38.259998,120.599998,8.20,...,19.299999,99.180000,36.810001,98.080002,96.000000,66.120003,39.889999,136.259995,22.959999,10580.190430
2019-01-03,42.939999,183.949997,81.209999,60.020000,71.510002,91.360001,69.050003,36.630001,121.400002,8.09,...,19.525000,95.459999,35.700001,95.449997,92.599998,64.459999,40.000000,134.759995,23.969999,10416.660156
2019-01-04,45.549999,191.050003,85.120003,63.189999,73.750000,92.180000,71.709999,38.599998,126.000000,8.52,...,19.924999,98.739998,36.669998,99.190002,95.400002,66.540001,40.400002,140.479996,24.760000,10767.690430
2019-01-07,45.150002,189.699997,83.559998,62.900002,74.699997,91.279999,72.120003,38.450001,126.400002,8.70,...,19.900000,99.709999,36.224998,97.889999,99.199997,66.620003,40.580002,140.639999,26.200001,10747.809570


In [5]:
# check if any NAs 
df.isna().sum()

DAX                       19
Linde                     19
SAP                        8
Deutsche Telekom          19
Volkswagen                26
Siemens                   19
Merck                     19
Airbus                    19
Mercedes Benz             19
Bayer                     26
Siemens Healthineers      19
Deutsche Post             19
BASF                      19
Infineon                  19
Deutsche Börse            26
RWE                       19
Henkel                    19
Adidas                    19
Sartorius                 19
Beiersdorf                19
Porsche                   26
E.ON                      19
Deutsche Bank             19
Vonovia                   19
Fresenius                 19
Symrise                   19
Continental               19
Delivery Hero             26
Brenntag                  19
Qiagen                    26
Fresenius Medical Care    19
Siemens Energy            19
HeidelbergCement          19
Puma                      19
MTU Aero Engin

In [6]:
df = df.interpolate(method='time', limit=7).fillna(value=None, method='bfill', axis=0, inplace=False, limit=7, downcast=None)

In [7]:
# check if any NAs left
df.isna().sum()

DAX                       0
Linde                     0
SAP                       0
Deutsche Telekom          0
Volkswagen                0
Siemens                   0
Merck                     0
Airbus                    0
Mercedes Benz             0
Bayer                     0
Siemens Healthineers      0
Deutsche Post             0
BASF                      0
Infineon                  0
Deutsche Börse            0
RWE                       0
Henkel                    0
Adidas                    0
Sartorius                 0
Beiersdorf                0
Porsche                   0
E.ON                      0
Deutsche Bank             0
Vonovia                   0
Fresenius                 0
Symrise                   0
Continental               0
Delivery Hero             0
Brenntag                  0
Qiagen                    0
Fresenius Medical Care    0
Siemens Energy            0
HeidelbergCement          0
Puma                      0
MTU Aero Engines          0
Covestro            

In [8]:
df.head()

,DAX,Linde,SAP,Deutsche Telekom,Volkswagen,Siemens,Merck,Airbus,Mercedes Benz,Bayer,...,Brenntag,Qiagen,Fresenius Medical Care,Siemens Energy,HeidelbergCement,Puma,MTU Aero Engines,Covestro,Zalando,HelloFresh
Date,,,,,,,,,,,,,,,,,,,,,
2018-12-31,42.900002,184.399994,84.169998,60.720001,69.500000,91.860001,69.739998,38.259998,120.599998,8.15,...,19.299999,99.550003,36.810001,98.080002,96.000000,66.120003,39.889999,136.259995,22.959999,10580.190430
2019-01-02,42.900002,184.399994,84.169998,60.720001,69.599998,91.860001,69.739998,38.259998,120.599998,8.20,...,19.299999,99.180000,36.810001,98.080002,96.000000,66.120003,39.889999,136.259995,22.959999,10580.190430
2019-01-03,42.939999,183.949997,81.209999,60.020000,71.510002,91.360001,69.050003,36.630001,121.400002,8.09,...,19.525000,95.459999,35.700001,95.449997,92.599998,64.459999,40.000000,134.759995,23.969999,10416.660156
2019-01-04,45.549999,191.050003,85.120003,63.189999,73.750000,92.180000,71.709999,38.599998,126.000000,8.52,...,19.924999,98.739998,36.669998,99.190002,95.400002,66.540001,40.400002,140.479996,24.760000,10767.690430
2019-01-07,45.150002,189.699997,83.559998,62.900002,74.699997,91.279999,72.120003,38.450001,126.400002,8.70,...,19.900000,99.709999,36.224998,97.889999,99.199997,66.620003,40.580002,140.639999,26.200001,10747.809570


In [9]:
# plot daily portfolio worth
fig = px.line(df, 
              y="DAX", 
              title='DAX')
fig.show()

In [10]:
df.mean()

DAX                          45.470285
Linde                       250.640084
SAP                         102.401960
Deutsche Telekom             59.600831
Volkswagen                   65.039448
Siemens                      97.254723
Merck                        72.840995
Airbus                       59.750365
Mercedes Benz               103.355129
Bayer                        10.040927
Siemens Healthineers         74.930464
Deutsche Post                39.087090
BASF                         15.800856
Infineon                      9.788117
Deutsche Börse               36.296891
RWE                          40.363358
Henkel                       60.669920
Adidas                       82.784616
Sartorius                    41.340727
Beiersdorf                   25.812226
Porsche                     247.449909
E.ON                         56.405647
Deutsche Bank               137.776979
Vonovia                     195.695770
Fresenius                   233.499184
Symrise                  

In [11]:
# normalized daily returns
def normalized_returns(df):
    return np.log(1 + df.pct_change(periods=1).fillna(value=None, method='bfill', axis=0, inplace=False, limit=7, downcast=None)) 

# create weights
def weight_creator(df):
    rand = np.random.random(len(df.columns))
    rand /= rand.sum()
    return rand

# calculate portfolio return
def portfolio_returns(df, weights):
    return np.dot(df.mean(), weights)

# calculate portfolios standard deviation
def portfolio_std(df, weights):
    return (np.dot(np.dot(df.cov(), weights), weights))**(1/2)*np.sqrt(250)

In [12]:
# plot daily portfolio worth
fig = px.line(normalized_returns(df), 
              y="DAX", 
              title='DAX: Normalized Returns')
fig.show()

In [13]:
# testing the functions
df_returns = normalized_returns(df)
weights = weight_creator(df_returns)
sdev = portfolio_std(df_returns, weights)
returns = portfolio_returns(df_returns, weights)

print('portfolio return: {:.5f}'.format(returns))
print('portfolio standard deviation: {:.2f}'.format(sdev))

portfolio return: 0.00009
portfolio standard deviation: 0.21


In [14]:
# Monte-Carlo Simulation

returns = []
stds = []
w = []

df_returns = normalized_returns(df)

for n in range(1000):
    weights = weight_creator(df_returns)
    w.append(weights)
    returns.append(portfolio_returns(df_returns, weights))
    stds.append(portfolio_std(df_returns, weights))

In [17]:
# plot returns vs risk
fig = px.scatter(x=np.array(stds), y=np.array(returns), 
              title='DAX')
fig.show()

TypeError: Object of type function is not JSON serializable